In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

In [2]:
%%sql
SELECT * FROM mapping_categories WHERE categorie='nitrite'

Running query in 'duckdb:///../../database/data.duckdb'

cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
NO3,1340,NITRATES (EN NO3),Nitrates (en NO3),None,nitrite
NO3_NO2,6374,NITRATES/50 + NITRITES/3,Nitrates/50 + Nitrites/3,None,nitrite
NO2,1339,NITRITES (EN NO2),Nitrites (en NO2),None,nitrite
NO3,1340,NITRATES (EN NO3),Nitrates (en NO3),14797-55-8,nitrite
NO2,1339,NITRITES (EN NO2),Nitrites (en NO2),14797-65-0,nitrite


In [6]:
%%sql --save int__resultats_nitrites_udi_dernier
WITH
last_pvl AS (
    SELECT
        cdreseau,
        categorie,
        cdparametresiseeaux,
        datetimeprel,
        valtraduite,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau, cdparametresiseeaux
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
        int__resultats_udi_communes
    WHERE
        categorie = 'nitrite'
        AND
        CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
),

split_nitrites AS (
    SELECT
        last_pvl.cdreseau,
        last_pvl.categorie,
        COUNT(DISTINCT last_pvl.cdparametresiseeaux) AS nb_parametres,
        MAX(last_pvl.datetimeprel) AS dernier_prel_datetime,
        MAX(CASE  WHEN last_pvl.cdparametresiseeaux = 'NO3' THEN last_pvl.valtraduite END) AS valtraduite_no3,
        MAX(CASE  WHEN last_pvl.cdparametresiseeaux = 'NO3_NO2' THEN last_pvl.valtraduite END) AS valtraduite_no3_no2,
    FROM
        last_pvl
    WHERE
        last_pvl.row_number = 1
    GROUP BY  
        last_pvl.cdreseau,
        last_pvl.categorie        
)

SELECT
    split_nitrites.cdreseau,
    split_nitrites.categorie,
    'dernier_prel' AS periode,
    split_nitrites.dernier_prel_datetime,
    split_nitrites.nb_parametres,
    CASE
        WHEN
            split_nitrites.valtraduite_no3 IS NULL 
            OR split_nitrites.valtraduite_no3_no2 IS NULL
            OR split_nitrites.valtraduite_no3 = 0
            OR split_nitrites.valtraduite_no3_no2 = 0
            THEN 'non_quantifie'
        WHEN
            split_nitrites.valtraduite_no3 <0.5
            AND split_nitrites.valtraduite_no3_no2 <1
            THEN 'conforme'
        WHEN
            split_nitrites.valtraduite_no3 >= 0.5
            OR split_nitrites.valtraduite_no3_no2 >= 1
            THEN 'non_conforme'
        ELSE 'error'
    END AS resultat
FROM
    split_nitrites



Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,resultat
001000260,nitrite,dernier_prel,2024-12-06 12:02:00,3,non_conforme
001000268,nitrite,dernier_prel,2024-11-28 08:25:00,3,non_conforme
001000275,nitrite,dernier_prel,2024-07-17 09:30:00,3,non_quantifie
001000293,nitrite,dernier_prel,2024-10-10 10:20:00,3,non_conforme
001000374,nitrite,dernier_prel,2024-05-22 09:40:00,3,non_conforme
001000539,nitrite,dernier_prel,2024-12-09 15:22:00,3,non_conforme
001000541,nitrite,dernier_prel,2024-12-03 11:55:00,3,non_conforme
001000578,nitrite,dernier_prel,2024-11-29 08:45:00,3,non_conforme
001000792,nitrite,dernier_prel,2024-11-21 10:12:00,3,non_conforme
001000846,nitrite,dernier_prel,2024-11-22 10:53:00,3,non_conforme


In [7]:
%%sql --with int__resultats_nitrites_udi_dernier
SELECT 
    * 
FROM 
    int__resultats_nitrites_udi_dernier
WHERE 
    resultat='error'

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,resultat


# Check errors

In [5]:
%%sql --save int__resultats_nitrites_udi_dernier
WITH
last_pvl AS (
    SELECT
        cdreseau,
        categorie,
        cdparametresiseeaux,
        datetimeprel,
        valtraduite,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau, cdparametresiseeaux
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
        int__resultats_udi_communes
    WHERE
        categorie = 'nitrite'
        AND
        CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
)
    SELECT
        last_pvl.cdreseau,
        last_pvl.categorie,
        COUNT(DISTINCT last_pvl.cdparametresiseeaux) AS nb_parametres,
        MAX(last_pvl.datetimeprel) AS dernier_prel_datetime,
        MAX(CASE  WHEN last_pvl.cdparametresiseeaux = 'NO3' THEN last_pvl.valtraduite END) AS valtraduite_no3,
        MAX(CASE  WHEN last_pvl.cdparametresiseeaux = 'NO3_NO2' THEN last_pvl.valtraduite END) AS valtraduite_no3_no2,
    FROM
        last_pvl
    WHERE
        last_pvl.row_number = 1
        AND cdreseau IN ('088002296','073002059','973000028')
    GROUP BY  
        last_pvl.cdreseau,
        last_pvl.categorie        


Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,nb_parametres,dernier_prel_datetime,valtraduite_no3,valtraduite_no3_no2
973000028,nitrite,2,2024-12-02 11:30:00,0.000,None
088002296,nitrite,1,2024-09-27 12:00:00,None,None
073002059,nitrite,1,2024-10-23 09:28:00,None,None
